In [ ]:
SAVE_NAME = 'MRM_E200_5epoch'

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [3]:
# Helper function
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()

In [4]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Load numpy array

In [5]:
all_npy = np.load('./npy/all_4876.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre.npy')

print('All features:', all_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

All features: (165, 4876)
Movie genre: (165, 20)
User following: (1582, 165)
User genre: (1582, 20)


In [6]:
usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies
print(usr_nb, movie_nb)

usr_test_amount = 150
movie_test_amount = 32
print(usr_test_amount, movie_test_amount)

latent_dim = 128 # latent dims
ft_dim = all_npy.shape[1] # feature dims
embedding_dims = 200
print(latent_dim, ft_dim, embedding_dims)

1582 165
150 32
128 4876 200


### Normalize usr_genre

In [7]:
usr_genre_norm = np.zeros(usr_genre.shape)
for i in range(len(usr_genre)):
    usr_genre_norm[i] = usr_genre[i]/np.max(usr_genre[i])
print(usr_genre_norm.shape)
# print('Before:', usr_genre)
# print('After:', usr_genre_norm)

(1582, 20)


# Training & testing split

## Prepare

In [8]:
#The number of followers for each movie
moive_followers = np.sum(usr_following, axis=0)
# print(moive_followers)

print('Min number of followers:', np.min(moive_followers))
print('Max number of followers:', np.max(moive_followers))
print('Avg of followers:', np.mean(moive_followers))

asc = np.sort(moive_followers)
# print(asc)
desc = np.flip(asc)
# print(desc)

over5 = 0
for num in moive_followers:
    if num >= 5:
        over5 += 1
print('The num of followers over 5:', over5)

Min number of followers: 1
Max number of followers: 520
Avg of followers: 142.0969696969697
The num of followers over 5: 163


In [9]:
print('Over 50:', np.sum(moive_followers >= 50))
print('Over 100:', np.sum(moive_followers >= 100))
print('Over 150:', np.sum(moive_followers >= 150))
print('Over 200:', np.sum(moive_followers >= 200))
print('Over 250:', np.sum(moive_followers >= 250))
print('Over 300:', np.sum(moive_followers >= 300))

Over 50: 125
Over 100: 89
Over 150: 58
Over 200: 42
Over 250: 31
Over 300: 21


In [10]:
over200_idx = np.nonzero(moive_followers >= 200)[0]
print(over200_idx.shape, over200_idx)

random.seed(42)
movie_test_idx = sorted(random.sample(list(over200_idx), movie_test_amount))
print(len(movie_test_idx), movie_test_idx) # 32 [0, 2, 3, 12, 24, 28, 30, 44, 49, 55, 57, 58, 60, 66, 78, 80, 81, 84, 86, 87, 102, 112, 119, 122, 123, 125, 127, 128, 129, 144, 161, 164]

(42,) [  0   2   3   4   9  12  24  28  30  34  40  44  49  55  57  58  60  66
  68  78  80  81  84  86  87  99 101 102 112 119 122 123 125 126 127 128
 129 134 144 156 161 164]
32 [0, 2, 3, 12, 24, 28, 30, 44, 49, 55, 57, 58, 60, 66, 78, 80, 81, 84, 86, 87, 102, 112, 119, 122, 123, 125, 127, 128, 129, 144, 161, 164]


In [11]:
#The number of following movie for each user
each_user = np.sum(usr_following, axis=1)
# print(each_user)

print('Min number of followings:', np.min(each_user))
print('Max number of followings:', np.max(each_user))
print('Avg of followers:', np.mean(each_user))

asc = np.sort(each_user)
# print(each_user)
# print(asc)
desc = np.flip(asc)
# print(desc)

Min number of followings: 10
Max number of followings: 133
Avg of followers: 14.820480404551201


In [12]:
print('Over 10:', np.sum(each_user >= 10))
print('Over 12:', np.sum(each_user >= 12))
print('Over 14:', np.sum(each_user >= 14))
print('Over 16:', np.sum(each_user >= 16))
print('Over 18:', np.sum(each_user >= 18))
print('Over 20:', np.sum(each_user >= 20))

Over 10: 1582
Over 12: 937
Over 14: 613
Over 16: 440
Over 18: 315
Over 20: 229


In [13]:
usr_idx = [i for i in range(len(usr_following))]
print(len(usr_idx))

random.seed(42)
test_idx = sorted(random.sample(usr_idx, usr_test_amount))
print(len(test_idx), test_idx[:10]) # 150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]

1582
150 [13, 51, 54, 61, 65, 88, 93, 96, 114, 130]


## Setup 

In [14]:
# init
train_t = []
train_f = []
test_t = []
test_f = []

for i in range(usr_nb):
    # init
    t_for_train = []
    f_for_train = []
    t_for_test = []
    f_for_test = []
    
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(movie_nb):
            if usr_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
                
        train_t.append(t_for_train)
        train_f.append(f_for_train)
#         print(len(t_for_train) + len(f_for_train))
        
    else: #if in test id, choose half of true and other 
        temp_t = []
        temp_f = []
        
        for j in range(movie_nb):
            if usr_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        
        # random choose half true and half false for test 
        t_for_test = random.sample(temp_t, math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f, movie_test_amount-len(t_for_test))
        
        test_t.append(t_for_test)
        test_f.append(f_for_test)
        
        #the others for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t.append(t_for_train)
        train_f.append(f_for_train)
        
    if not (len(t_for_train) + len(f_for_train) + len(t_for_test) + len(f_for_test)) == movie_nb:
        print('Error!!!')
        break

In [15]:
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_f))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1582
The length of train_f: 1582
The length of test_t: 150
The length of test_f: 150


## Stat

In [16]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train / usr_nb
print('Training:', avg)

#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test / usr_test_amount
print('Testing:', avg)

Training: 14.139064475347661
Testing: 7.1866666666666665


In [17]:
all_auxilary = [i for i in range(movie_nb)]

# Recommendation model

In [18]:
tf.reset_default_graph()

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None, ft_dim))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))
positive_id = tf.placeholder(tf.int32, shape=(None,))
positive_len = tf.placeholder(tf.int32,shape=(1,))

image_i = tf.placeholder(tf.float32, [1, ft_dim])
image_j = tf.placeholder(tf.float32, [1, ft_dim])

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [usr_nb, latent_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [movie_nb, latent_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [movie_nb, latent_dim],
                               initializer=tf.random_normal_initializer(0,0.1,seed=3))
    
#     W1 = tf.get_variable("W1", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [usr_nb, movie_nb, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [usr_nb, movie_nb, embedding_dims], initializer=tf.contrib.layers.xavier_initializer())
#     Wve = tf.get_variable("Wve", [embedding_dims, ft_dim], initializer=tf.contrib.layers.xavier_initializer())
    
    aux_new = tf.get_variable("aux_new", [1, latent_dim], initializer=tf.constant_initializer(0.0))

with tf.variable_scope('feature_level'):
    embedding = tf.get_variable("embedding", [embedding_dims,ft_dim],
                                initializer=tf.contrib.layers.xavier_initializer())
    Beta = tf.get_variable("beta", [usr_nb, embedding_dims],
                           initializer=tf.random_normal_initializer(0.01, 0.001, seed=10))
    
#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user)
vi = tf.nn.embedding_lookup(item_latent, i)
vj = tf.nn.embedding_lookup(item_latent, j)

# w1 = tf.nn.embedding_lookup(W1, user)
wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user))
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user))
wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user))
wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user))

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [19]:
a_list = tf.Variable([])
q = tf.constant(0)

def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])

def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
    
    a_list = tf.concat([a_list,[(tf.nn.relu(tf.matmul(wuui, u, transpose_b=True) +
                                            tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                            tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                            tf.matmul(wvui, tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q, a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

a_list_smooth = tf.add(a_list,0.0000000001)
a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

# norm_par = [wu,wy,wa,wv]
norm_par = [tf.reduce_sum(tf.multiply(u, u)),
            tf.reduce_sum(tf.multiply(vi, vi)),
            tf.reduce_sum(tf.multiply(vj, vj)),
            tf.reduce_sum(tf.multiply(wu, wu)),
            tf.reduce_sum(tf.multiply(wy, wy)),
            tf.reduce_sum(tf.multiply(wa, wa)),
            tf.reduce_sum(tf.multiply(wv,wv)),
            tf.reduce_sum(tf.multiply(beta,beta)),
            tf.reduce_sum(tf.multiply(embedding,embedding))]

wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))

last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

aux_np = tf.expand_dims(tf.zeros(latent_dim),0)
q = tf.constant(0)

def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_, aux_np = tf.while_loop(sum_att_cond, sum_att_body, [q,aux_np])

aux_part = tf.matmul(aux_np, vi, transpose_b=True)
aux_np += u #user_latent factor + sum (alpha*auxilary)
aux_new = tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

#矩陣中對應函數各自相乘
# ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

xuij = tf.subtract(xui,xuj)

l2_norm = tf.add_n([
            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.01 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.01 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.01 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.001 * tf.reduce_sum(tf.multiply(beta,beta)),
            0.01 * tf.reduce_sum(tf.multiply(embedding,embedding))
          ])

loss = l2_norm - tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


## Training

In [20]:
print('Start time:', time.ctime())

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0 = time.time()

train_yes_id=[]

for q in range(5):
    print('Epoch:',q)
    train_auc = 0
    total_loss = 0
    xuij_auc = 0
    length = 0
    
    for z in range(usr_nb):
        writeProgress('Progress:', z, usr_nb)
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes = []
        yesr = []
        
#         #選全部的Positive
#         sample = random.sample(train_t[z],len(train_t[z]))
        #選全部的電影
        sample = all_auxilary
        
        #change
        r_3 = np.zeros(len(sample))
         
        for b in range(len(sample)):
            yes.append(all_npy[sample[b]])
            yesr.append(movie_genre[sample[b]] * usr_genre_norm[z])
        
        for b in range(len(yesr)):
            r_3[b]=max(yesr[b])
        #print('r_3:',r_3)
        
        yes = np.array(yes)
        
        # positive sample
        train_t_sample = train_t[z]
        for ta in train_t_sample:
            #print(ta,'--> positive feedback')
            
            pos = sample.index(ta)
            
            image_1=np.expand_dims(all_npy[ta],0)
            train_f_sample = random.sample(train_f[z],10)
            
            for b in train_f_sample:
                image_2 = np.expand_dims(all_npy[b],0)
                
                _last_be_relu, _norm_par, _a_list, r3, _auc, _loss, _ = sess.run(
                    [last_be_relu, norm_par, a_list_smooth, a_list_soft, auc, loss, train_op], 
                    feed_dict={user: [z], i: [ta], j: [b], xf: yes, 
                               l_id:sample, l_id_len:[len(sample)],
                               positive_id: train_t[z], positive_len:[len(train_t[z])],
                               r: r_3, image_i: image_1, image_j: image_2})
                
#                 Observe all params
                print('u,vi,vj',_norm_par[:3])
                print('w1,wu,wy,wa,wv',_norm_par[3:7])
                print('beta',_norm_par[7])
                print('Embedding',_norm_par[8])

#                 print('after softmax:', r3)
#                 print('before softmax:', _a_list)
                print('---------------------------------------------------')
                
                train_auc += _auc
                total_loss += _loss
                length += 1
    
    print("{:<20}{}".format('total_loss', total_loss/length))
    print("{:<20}{}".format('train_auc:', train_auc/length))
    
    loss_acc_list.append([total_loss/length, train_auc/length])
    
    print('\tCurrent time:', time.ctime(), ' sec')
    print('==================================================')
    
print('Total cost time:',time.time()-t0, ' sec')

print('End time:', time.ctime())

Start time: Thu Mar 12 18:30:55 2020
Epoch: 0
u,vi,vj [1.2751031, 1.1690395, 0.94909155]
w1,wu,wy,wa,wv [0.09117579, 0.09104867, 0.09094404, 0.114014454]
beta 0.020226717
Embedding 384.43512
---------------------------------------------------
u,vi,vj [1.2731178, 1.1691359, 1.1646707]
w1,wu,wy,wa,wv [0.08384072, 0.0837325, 0.08362244, 0.10380146]
beta 0.020196242
Embedding 383.25494
---------------------------------------------------
u,vi,vj [1.271124, 1.1692345, 1.294842]
w1,wu,wy,wa,wv [0.076904126, 0.07681458, 0.07669917, 0.09420821]
beta 0.020210655
Embedding 382.1132
---------------------------------------------------
u,vi,vj [1.2691281, 1.1693344, 1.018029]
w1,wu,wy,wa,wv [0.070372395, 0.07030124, 0.070180744, 0.08524296]
beta 0.020244295
Embedding 380.9077
---------------------------------------------------
u,vi,vj [1.2671316, 1.1694367, 1.3640485]
w1,wu,wy,wa,wv [0.06424061, 0.06418744, 0.06406262, 0.076896034]
beta 0.020295177
Embedding 379.6981
--------------------------------

KeyboardInterrupt: 

In [ ]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('==================================================')

In [ ]:
# training history
epochs = range(1, len(loss_acc_list) + 1)
print(epochs)
loss = [ls[0].tolist()[0][0] for ls in loss_acc_list]
print(loss)
acc = [ls[1] for ls in loss_acc_list]
print(acc)

In [ ]:
# 繪製結果
import matplotlib.pyplot as plt

# plt.figure()

plt.plot(epochs, acc, 'b', label='Training acc')
plt.title('Training accuracy')
plt.legend()

plt.show()

# plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.title('Training loss')
plt.legend()

plt.show()

# Get latent factor and Each weight

In [ ]:
U, Y, A, E, Au, Ay, Aa, Av, B = sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv, Beta])

In [ ]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('Embedding shape:', E.shape)
print('Wu weight shape:', Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:', Aa.shape)
print('Wv weight shape:', Av.shape)
print('Beta shape:',B.shape)

In [ ]:
np.savez('./weight/' + SAVE_NAME + '.npz', 
         U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av, B=B)

In [ ]:
# # reload params if crash
# SAVE_NAME = 'MRM_ALL_Embedding200_L2_resplit'

# params = np.load('./weight/' + SAVE_NAME + '.npz')
# print(params)
# U = params['U']
# Y = params['Y']
# A = params['A']
# E = params['E']
# Au = params['Wu']
# Ay = params['Wy']
# Aa = params['Wa']
# Av = params['Wv']
# B = params['B']

# print('User latent shape: ',U.shape)
# print('photo latent shape: ', Y.shape)
# print('Auxilary latent shape: ',A.shape)
# print('Embedding shape:', E.shape)
# print('Wu weight shape:', Au.shape)
# print('Wy weight shape:', Ay.shape)
# print('Wa weight shape:', Aa.shape)
# print('Wv weight shape:', Av.shape)
# print('Beta shape:',B.shape)

# Testing Part

In [ ]:
'''
usr_test_amount = 150
movie_test_amount = 32
'''

#with Embedding
result = np.zeros((usr_test_amount, movie_nb))
RS = np.zeros((usr_test_amount, movie_nb))

#test_idx --> Test 的 index length = 150
test_yes_id = []

for s in range(usr_test_amount):
    print(s, test_idx[s])

    yes = []
    sample = random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]]))
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha = np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r = np.max(movie_genre[sample[a]] * usr_genre_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        
# #         ''' Observe each part in attention
#         WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
#         WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
#         WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
#         WvVy = np.sum(np.dot(np.dot(Av[test_idx[s]], E),np.expand_dims(all_npy[sample[a]],0).T))
#         print('The sum of each par -->',
#               '\nw1:',testW1,
#               '\nWuU:',WuUu,
#               '\nwyY:',WyYy,
#               '\nWaA:',WaAa,
#               '\nWvV:',WvVy)
# #         '''
        
        alpha[a] = np.sum((relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T) +
                                np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T) +
                                np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T) +
                                np.dot(np.dot(Av[test_idx[s]], E),np.expand_dims(all_npy[sample[a]],0).T))))*r
        
    mul = np.zeros((1,latent_dim))
    
    print("{:<15}{}".format('alpha:', alpha))
    print("{:<15}{}".format('softmax alpha:', softmax(alpha)))
    print('==================================================')
    
    for i in range(len(sample)):
        mul += alpha[i] * A[sample[i]] #attention alpha * Ai part 
    new_mul = mul + U[test_idx[s]]  #(U+auxilary)
    
    for k in range(movie_nb):
        result[s][k] = np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T) + np.dot(B[test_idx[s]], np.dot(E, all_npy[k].T))

In [ ]:
#取出test的資料
print(RS.shape)

testRS = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32
target = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32
        
for z in range(usr_test_amount):
    user_id = test_idx[z]
    # positive target YouTuber list
    youtube_t = test_t[z] 
    # not target YouTuber list
    youtube_f = test_f[z]
    
#     print(user_id)
#     print(youtube_t)
#     print(youtube_f)
    
    #前面放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
        
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
    
#     print(testRS[z])
#     print(target[z])
#     print('==============================')

In [ ]:
print(target.shape, testRS.shape)
sumtarget = np.sum(target)
print('num of positive data in testing:', sumtarget) # whole matrix: 4800

# Metrics

## Top N

In [ ]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

def topN(RSls, n):
    maxn = np.argsort(RSls)[::-1][:n]
    return maxn

## NDCG

In [ ]:
all_sort = []

for i in range(usr_test_amount):
    all_sort.append(topN(list(testRS[i]),len(testRS[i])))
    
all_sort = np.asarray(all_sort)
print(all_sort.shape)

In [ ]:
def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg = 0
    for i in range(len(prec_list)):
        dcg += (2**prec_list[i]-1)/math.log2(i+2)
    return dcg

def NDCG(target, testRS, num_ndcg): #target是真正的喜好
    total_ndcg = 0
    
    for m in range(usr_test_amount): # the number of testing users
        idcg = DCG(target[m][:num_ndcg])
        
        pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        total_ndcg += ndcg
        
    avg_ndcg = total_ndcg/usr_test_amount
    return avg_ndcg

## MAP

In [ ]:
from sklearn.metrics import average_precision_score

def MAP(target,testRS):
    total_prec = 0
    for u in range(usr_test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec += average_precision_score(y_true, y_scores)
        
    Map_value = total_prec/usr_test_amount
    
    return Map_value

In [ ]:
# Top N
N = [1, 3, 5]
correct = 0

for n in N:
    print('Top', n)
    
    for i in range(len(testRS)):
        topn = topN(testRS[i], n)
        sum_target = int(np.sum(target[i]))
        
        TP = 0
        for i in topn:
            if i < sum_target:
                TP += 1
                
        correct += TP

    print('Num of TP:', correct)

    prec = correct/(len(testRS)*n)
    recall = correct/sumtarget
    
    print('prec:', prec)
    print('recall:', recall)
    print('F1_score:', F1_score(prec, recall))
    
    print('*****')

print('\n==============================\n')

# NDCG
num_ndcgs = [1, 3, 5, 10, 15, 20]
for num_ndcg in num_ndcgs:
    print('NDCG@', num_ndcg)
    print('NDCG score:', NDCG(target, testRS, num_ndcg))
    print('*****')

print('\n==============================\n')

# MAP
print('MAP:', MAP(target,testRS))